# Defining Tools

In [ ]:
from guidance import Tool
from typing import Literal
import random

def get_weather(city: str, unit: Literal["celsius", "fahrenheit"] = "celsius") -> str:
    """
    Get the current weather for a given city.
    
    Args:
        city (str): The name of the city to get the weather for.
        unit (Literal["celsius", "fahrenheit"]): The unit of temperature to return.
    
    Returns:
        str: A string describing the current weather in the specified city.
    """
    temp = random.randint(-10, 35)
    if unit == "fahrenheit":
        temp = temp * 9/5 + 32
    return f"The current temperature in {city} is {temp} degrees {unit}."

A `Tool` consist of:
1. A `callable` that should be invoked when the tool is called
2. A `name` that uniquely identifies the callable
3. A `description` that indicates the usage of the callable
4. A `schema` that specifies the types of the callable's arguments, along with any metadata such as examples, default values, etc.

Note that the `schema` currently must be a `pydantic.BaseModel`, but direct support for JSON schemas is incoming.

In [ ]:
from pydantic import BaseModel, Field, ConfigDict

class GetWeatherArgs(BaseModel):
    city: str = Field(..., description="The name of the city to get the weather for.")
    unit: Literal["celsius", "fahrenheit"] = Field("celsius", description="The unit of temperature to return.")

    model_config = ConfigDict(
        extra="forbid", # Disallow extra fields -- required for OpenAI
    )

tool = Tool(
    callable=get_weather,
    name="get_weather",
    description="Get the current weather for a given city.",
    schema=GetWeatherArgs
)

print(f"""\
Tool name:
{tool.name}

Tool description:
{tool.description}

Tool schema:
{tool.schema.model_json_schema()}\
""")

As an alternative to constructing a `Tool` manually, `guidance` can infer the tool's `name`, `description`, and `schema` from the callable's name, docstring, and signature annotations, respectively.

In [ ]:
tool = Tool.from_callable(get_weather)

print(f"""\
Tool name:
{tool.name}

Tool description:
{tool.description}

Tool schema:
{tool.schema.model_json_schema()}\
""")

# Calling Tools

In [ ]:
from guidance import gen, user, assistant

## Remote Models

Most remote models have native support for tool-calling.

With these models, you can simply pass a list of tools to `gen`.

Behind the scenes, `guidance` automatically translates the `gen` to a call to the model provider's API, which in the case of OpenAI looks like:

```python
from openai import OpenAI
client = OpenAI(...)

completion = client.chat.completions.create(
    messages=[...],
    tools=[
        {
            "type": "function",
            "function": {
                "name": tool.name,
                "description": tool.description,
                "parameters": tool.schema.model_json_schema(),
                "strict": True,
            }
        },
        ...
    ]
)
```

*Note*: you may pass a list of `callable` instead, and the `Tool`s will be automatically constructed for you.

In [ ]:
from guidance.models import OpenAI
lm = OpenAI("gpt-4o-mini")

with user():
    lm += "What is the weather like in San Francisco?"
with assistant():
    lm += gen(tools=[tool])
with assistant():
    lm += gen()

You may notice that the tool call is wrapped in `<function={name}>...</function>` tags and the result is wrapped in `<function_result>..</function_result>` tags. This is purely for visualization purposes, as OpenAI uses a structured representation under the hood which we (somewhat arbitrarily) represent as a string above.

## Local Models

### The hard way

Because local models do not have "native" support for tool calls, we have to provide a little more information. 

First, we prompt our model with an explanation of the tools that it has access to as well as the syntax we expect it to use. While `guidance` will guarantee that the model follows the specified syntax, it is no replacement for proper prompting.

In [ ]:
from guidance.models import LlamaCpp
from guidance import system
from huggingface_hub import hf_hub_download
from json import dumps

model = LlamaCpp(
    hf_hub_download(
        repo_id="unsloth/Qwen3-0.6B-GGUF",
        filename="Qwen3-0.6B-BF16.gguf",
    ),
    n_ctx=4096,
)

with system():
    model += f"""\
# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{dumps({"name": tool.name, "description": tool.description, "arguments": tool.schema.model_json_schema()})}
</tools>

For each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:
<tool_call>
{{"name": <function-name>, "arguments": <args-json-object>}}
</tool_call>"""

In [ ]:
from guidance import lark

https://github.com/guidance-ai/llguidance/blob/main/docs/syntax.md#tool-calling

In [ ]:
schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string", "const": tool.name},
        "arguments": tool.schema.model_json_schema(),
    },
    "required": ["name", "arguments"],
    "additionalProperties": False
}

l = lark(f"""\
start: TEXT | tool_call
tool_call: tool_call_trigger "\\n" tool_call_body "\\n" </tool_call>
tool_call_trigger: TEXT <tool_call>
tool_call_body[capture]: %json {dumps({
    "type": "object",
    "properties": {
        "name": {"type": "string", "const": tool.name},
        "arguments": tool.schema.model_json_schema(),
    },
    "required": ["name", "arguments"],
    "additionalProperties": False
})}
TEXT: /(.|\\n)*/
""")

tool_grammar = lark(l)

In [ ]:
from guidance import optional, special_token, json
tool_grammar = gen() + optional(
    special_token("<tool_call>")
    + json(
        schema={
            "type": "object",
            "properties": {
                "name": {"type": "string", "const": tool.name},
                "arguments": tool.schema.model_json_schema(),
            },
            "required": ["name", "arguments"],
            "additionalProperties": False
        },
        name="tool_call",
    )
    + special_token("</tool_call>")
)

In [ ]:
from guidance import special_token
with user():
    model += "What is the weather like in San Francisco?"
with assistant():
    model += special_token("<think>") + gen(max_tokens=100) + special_token("</think>")
    model += tool_grammar

### The easy way

In [ ]:
from guidance import ToolCallHandler

class Qwen3ToolCallHandler(ToolCallHandler):
    expr = re.compile(r"<tool_call>\n(?P<call>\{(.|\n)*\})\n</tool_call>")

    def trigger(self):
        return "<tool_call>"

    def begin(self, tool_name: str) -> str:
        return "<tool_call>\n"

    def body(self, tool: Tool) -> GrammarNode:
        return json(
            schema={
                "type": "object",
                "properties": {
                    "name": {"type": "string", "const": tool.name},
                    "arguments": tool.schema.model_json_schema(),
                },
                "required": ["name", "arguments"],
                "additionalProperties": False,
            }
        )

    def end(self) -> str:
        return "\n</tool_call><|im_end|>\n"

    def parse_tool_calls(self, text: str) -> list[RawToolCall]:
        matches = self.expr.finditer(text)
        tool_calls = []
        for match in matches:
            call_data = loads(match.group("call"))
            tool_calls.append(RawToolCall(name=call_data["name"], args=call_data["arguments"]))
        return tool_calls

    def format_return_value(self, value: Any) -> str:
        return f"<|im_start|>user\n<tool_response>\n{dumps(value)}\n</tool_response>"

In [ ]:

lm = LlamaCpp(
    hf_hub_download(
        repo_id="unsloth/Qwen3-0.6B-GGUF",
        filename="Qwen3-0.6B-BF16.gguf",
    ),
    n_ctx=4096,
    tool_call_handler_cls=Qwen3ToolCallHandler
)

with system():
    lm += f"""\
# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{dumps({"name": tool.name, "description": tool.description, "arguments": tool.schema.model_json_schema()})}
</tools>

For each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:
<tool_call>
{{"name": <function-name>, "arguments": <args-json-object>}}
</tool_call>"""
with user():
    lm += "What is the weather like in San Francisco?"
with assistant():
    lm += gen(tools=[tool])
with assistant():
    lm += gen()